In [2]:
import torch
import main as m
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
from torchvision.transforms import PILToTensor#, tensor

In [3]:
mean = torch.tensor([0.485, 0.456, 0.406])
std = torch.tensor([0.229, 0.224, 0.225])

In [4]:
dt= m.DataAugmentationDINO(
  global_crops_scale=(0.8, 1),
  local_crops_scale=(0.4, 1),
  local_crops_number=2
)

In [5]:
imgs = [Image.open(f) for f in sorted(Path("venice").iterdir()) ]
imgs_t = [dt(PILToTensor()(im).permute(1,2,0).unsqueeze(0)) for im in imgs]

fig,ax = plt.subplots(ncols=len(imgs), figsize=(5*len(imgs), 5))
for i, im in enumerate(imgs_t):
  ax[i].imshow((im[0][0].permute(1,2,0) * std) + mean)
  ax[i].axis("off")
  

  
(type(imgs_t), len(imgs_t), "batch"), (type(imgs_t[0]), len(imgs_t[0]), "global, local, local"), ([t.shape for t in imgs_t[0]], "frames/c/h/w")  

In [6]:
# torch.load("checkpoint.pth")["teacher"].keys()

In [31]:
model = m.vits.vit_small(patch_size=16, drop_path_rate=0.1)

model = m.utils.MultiCropWrapper(
  model, 
  m.DINOHead(
      in_dim=model.embed_dim,
      out_dim=65536,
      use_bn=False
  )
)
model;
# model.load_state_dict(torch.load("checkpoint.pth")["teacher"])




In [32]:
global_imgs = torch.stack([gll[0] for gll in imgs_t])
global_imgs.shape

In [33]:
teacher_global, teacher_patches, attn, query, key = model([global_imgs], return_track=True)

In [34]:
(
  (teacher_global.shape, "f/b/head_dim"), # cls token of each image processed through dino head
  (teacher_patches.shape, "f/b/p/(embed_dim*n_heads)"), # patch tokens
  (attn.shape, "f/b/n_heads/(p+1)*(p+1)"), # attention maps 
  (query.shape, "f/b/n_heads/(p+1)/embed_dim"), # query vectors
  (key.shape, "f/b/n_heads/(p+1)/embed_dim") # query vectors
)

In [35]:
masked_img = m.utils.MOT(attn, query, key, teacher_patches, patch_size=16, images=[global_imgs], num_samples=6)

In [36]:
masked_img.shape, "num_samples/b/f/c/h/w"

In [37]:
ns, b, f, c,h,w = masked_img.shape

In [38]:
fig, ax =plt.subplots(ncols=b,nrows=ns, figsize=(5*b, 5*ns))

for i in range(ns):
  for j in range(b):
    ax[i,j].imshow((masked_img.detach()[i,j,0].permute(1,2,0) * std)+mean)
    ax[i,j].axis("off")
